## Using the Prolog engine

ProbLog contains a builtin Prolog-like engine that can be also be used as a standalone Prolog.

Unlike regular Prolog systems, each grounding of a query will occur only once as a result, even if their are multiple proofs.

The engine support cyclic programs.

In [ ]:
from problog.engine import DefaultEngine
from problog.logic import *

First, we initialize a Prolog model.
We can load it from a string.

In [28]:
from problog.program import PrologString
pl = PrologString("""
mother_child(trude, sally).
 
father_child(tom, sally).
father_child(tom, erica).
father_child(mike, tom).
 
sibling(X, Y) :- parent_child(Z, X), parent_child(Z, Y).
 
parent_child(X, Y) :- father_child(X, Y).
parent_child(X, Y) :- mother_child(X, Y).
""")

or we can construct it in Python

In [25]:
from problog.program import SimpleProgram

# Define the language of terms
mother_child = Term('mother_child')
father_child = Term('father_child')
sibling = Term('sibling')
parent_child = Term('parent_child')
X, Y, Z = map(Var, 'XYZ')
trude, sally, tom, erica, mike = map(Term, ['trude', 'sally', 'tom', 'erica', 'mike'])

# Define the program
pl = SimpleProgram()
pl += mother_child(trude, sally)
pl += father_child(tom, sally)
pl += father_child(tom, erica)
pl += father_child(mike, tom)
pl += sibling(X, Y) << (parent_child(Z, X) & parent_child(Z, Y))
pl += parent_child(X, Y) << father_child(X, Y)
pl += parent_child(X, Y) << mother_child(X, Y)


Next we initialize the engine, and we prepare the model for querying.
The second step is optional but recommended when we want to query the same model multiple times.

In [29]:
engine = DefaultEngine()
db = engine.prepare(pl)

We can now query the model.
The engine only supports queries that consist of a single Term.

The result of ``query`` is a list of tuples representing the arguments of the query term.
If the query fails, this list is empty.

In [40]:
query_term = sibling(tom, sally)
res = engine.query(db, query_term)
print ('%s? %s' % (query_term, bool(res)))

query_term = sibling(sally, erica)
res = engine.query(db, query_term)
print ('%s? %s' % (query_term, bool(res)))

sibling(tom,sally)? False
sibling(sally,erica)? True


Variables in the query should be replaced by ``None`` or a negative number.

In [51]:
query_term = sibling(None, None)
res = engine.query(db, query_term)

for args in res:
    print query_term(*args)

sibling(sally,sally)
sibling(sally,erica)
sibling(erica,sally)
sibling(erica,erica)
sibling(tom,tom)


In [42]:
query_term = Term('sibling', Term('sally'), None)
res = engine.query(db, query_term)

for args in res:
    print query_term(*args)

sibling(sally,sally)
sibling(sally,erica)


In [52]:
query_term = Term('sibling', -1, -1)
res = engine.query(db, query_term)

for args in res:
    print query_term(*args)

sibling(sally,sally)
sibling(erica,erica)
sibling(tom,tom)


In [57]:
print engine.ground_all(db, queries=[query_term])


Queries : 
* sibling(sally,sally) : 0
* sibling(erica,erica) : 0
* sibling(tom,tom) : 0

